In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd

# --- config ---
MODEL_PATH = "./mangosteen_volume_model_all_suggessions.h5"
TEST_DATA_DIR = "./DataTest/images"   
VOLUME_FILES_DIR = "./DataTest/labels"  
IMG_SIZE = 224

In [2]:
# --- helper: โหลดและ preprocess ภาพ ---
def load_and_preprocess_image(img_path, img_size=IMG_SIZE):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Warning: cv2.imread failed for {img_path}")
        return np.zeros((img_size, img_size, 3), dtype=np.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_size, img_size))
    img = img.astype('float32') / 255.0
    return img

# --- โหลดโมเดล ---
print(f"Loading model from {MODEL_PATH}...")
model = load_model(MODEL_PATH)
print("✅ Model loaded.")

Loading model from ./mangosteen_volume_model_all_suggessions.h5...


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = './mangosteen_volume_model_all_suggessions.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# --- โหลด test data และ volume จริง ---
test_images = []
test_volumes = []
test_filenames = []

for filename in os.listdir(TEST_DATA_DIR):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(TEST_DATA_DIR, filename)
        base = filename.split('.', 1)[0]
        vol_file = os.path.join(VOLUME_FILES_DIR, base + ".txt")
        if os.path.exists(vol_file):
            try:
                with open(vol_file, 'r') as f:
                    vol = float(f.read().strip())
                test_images.append(load_and_preprocess_image(img_path))
                test_volumes.append(vol)
                test_filenames.append(filename)
            except Exception as e:
                print(f"⚠️ Skipping {filename}: {e}")
        else:
            print(f"⚠️ No volume file for {filename}")

if not test_images:
    raise SystemExit("❌ No test data found.")

X_test = np.stack(test_images, axis=0)
y_true = np.array(test_volumes, dtype='float32')
print(f"Loaded {len(X_test)} test images.")


Loaded 85 test images.


In [ ]:
# --- predict ---
print("Predicting volumes...")
y_pred = model.predict(X_test, batch_size=32).flatten()


Predicting volumes...
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step


In [ ]:
# --- evaluate ---
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
print(f"\nValidation results on unseen data:")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")



Validation results on unseen data:
MAE: 9.5989
MSE: 167.0126


In [ ]:
# --- save predictions ---
df_pred = pd.DataFrame({
    'filename': test_filenames,
    'true_volume': y_true,
    'predicted_volume': y_pred
})
df_pred.to_csv("predicted_volumes_with_labels.csv", index=False)
print("✅ Predictions saved to predicted_volumes_with_labels.csv")

✅ Predictions saved to predicted_volumes_with_labels.csv
